In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential, losses, metrics, activations
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with open("download-cats", "w") as f:
  f.write("""
  if [ ! -d FACES128x128 ]; then
    wget "https://www.dropbox.com/s/mm1bwgfk65cxtv3/FACES128x128.zip?dl=1" -O "FACES128x128.zip"
    unzip "FACES128x128.zip" > /dev/null
    rm "FACES128x128.zip"
  fi
  """)
!bash download-cats

In [ ]:
 def normalize(x):
  # Normalizes to [-1, 1]
  return (x - 127.5) / 127.5

 def load_image(path, size = None):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image)
  image = tf.cast(image, tf.float32)
  image = tf.reshape(image, [128, 128, 3])
  image = normalize(image)
  if size is not None:
    image = tf.image.resize(image, (size, size))

  return image

In [ ]:
TRAINING_SET_SIZE = 10_000
BATCH_SIZE = 64
IMAGE_SIZE = 64 # Max 128

dataset = tf.data.Dataset.list_files("FACES128x128/*.jpg")
dataset = dataset \
    .map(lambda path: load_image(path, IMAGE_SIZE), num_parallel_calls=tf.data.experimental.AUTOTUNE) \
    .shuffle(buffer_size = 50_000) \
    .batch(BATCH_SIZE, drop_remainder=True)



In [ ]:
def to_image(img):
  img = img.reshape([IMAGE_SIZE, IMAGE_SIZE, 3])
  img = img * 127.5 + 127.5
  img = img.astype("uint8")
  return img

def show_image(img):
  img = to_image(img)
  plt.imshow(img)
  plt.show()

sample = next(iter(dataset)).numpy()[0]
show_image(sample)

In [ ]:
# Definer diskriminator-modellen. I motsetning til klassifikatoren skal denne gi
# ut én output-verdi: hvor vidt bilde-input-en er autentisk (true) eller 
# generert (false). 

discriminator = None

discriminator.compile(loss = losses.BinaryCrossentropy(from_logits=True))
discriminator.summary()

In [ ]:
noise_dim = 100

# Definer generator-modellen. Denne skal ta inn en støy-vektor (en vektor med 
# størrelse lik noise_dim av tilfeldige flyttall) og gi ut et bilde.

generator = None

generator.compile(loss = losses.BinaryCrossentropy(from_logits = True))
generator.summary()


In [ ]:
def generate_noise(dim, batch_size = 1):
  return tf.random.normal([batch_size, dim])

# Generer og viser et bilde fra den utrente generatoren

generated_image = generator(generate_noise(noise_dim)).numpy()

show_image(generated_image)


In [ ]:
# Her kan du bruke samme train_step som i forrige notebook

@tf.function
def train_step(real_images):
  
  D_loss = None
  G_loss = None
  P_real = None
  P_fake = None

  return D_loss, G_loss, P_real, P_fake



In [ ]:
def show_image_grid(images):
  N = images.shape[0]

  fig = plt.figure(figsize=(3, 3))
  fig.set_size_inches(10, 10)

  for i in range(N):
      plt.subplot(3, 3, i+1)
      img = to_image(images[i])
      plt.imshow(img)
      plt.axis('off')

  plt.show()

In [ ]:
# Tips og triks
#
# Er diskriminatoren mye sterkere enn generatoren (dette ser du hvis 
# diskriminatoren konsekvent vurderer både ekte og falske bilder med
# høy treffsikkerhet)?
# - Kan f.eks. kompensere med en lavere learning rate for diskriminatoren. 

In [ ]:
EPOCHS = 50

noise_for_training_visualization = generate_noise(noise_dim, batch_size = 9)

for epoch in range(EPOCHS):

  print("Epoch", epoch),

  for step, batch in enumerate(dataset):
    Ld, Lg, Pr, Pf = train_step(batch)

    if step % 10 == 0:
      print("Step {}. Ld={}, Lg={}, Pr={}, Pf={}".format(step, Ld, Lg, Pr, Pf))

  show_image_grid(generator(noise_for_training_visualization).numpy())


  

In [ ]:
# Denne kan du bruke til å utforske generatoren!

from ipywidgets import interact

p1 = generate_noise(noise_dim)
p2 = generate_noise(noise_dim)
p3 = generate_noise(noise_dim)

@interact(k1=(0, 1.0), k2=(0, 1.0))
def g(k1, k2):
    p = p1 + k1 * (p2 - p1) + k2 * (p3 - p1)
    show_image(generator(p).numpy())